In [1]:
from pyspark import SparkConf, SparkContext
# 스파크 환경 설정 객체 생성
conf = SparkConf().setMaster("local").setAppName("mnm-rdd-count")
spark = SparkContext(conf=conf).getOrCreate()
spark

24/12/04 11:33:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


<SparkContext master=local appName=mnm-rdd-count>

In [7]:
foods = spark.parallelize([ "파스타", "피자", "샐러드", "서브웨이", "닭갈비"])
foods

ParallelCollectionRDD[3] at readRDDFromFile at PythonRDD.scala:274

In [5]:
foods.collect()

['파스타', '라면', '샐러드', '서브웨이', '닭갈비']

In [6]:
foods.countByValue()

defaultdict(int, {'파스타': 1, '라면': 1, '샐러드': 1, '서브웨이': 1, '닭갈비': 1})

In [9]:
#상위 3개만 가져와
foods.take(3)

['파스타', '피자', '샐러드']

In [10]:
foods.first()

'파스타'

In [12]:
# RDD 갯수
foods.count()

5

In [15]:
#중복 제거한 목록
#Transformation 연산
fd_dist = foods.distinct()
fd_dist

PythonRDD[16] at RDD at PythonRDD.scala:53

In [16]:
fd_dist.collect()

['파스타', '피자', '샐러드', '서브웨이', '닭갈비']

In [ ]:
foods.foreach(lambda x : print(x))

In [17]:
#워커너드에서 실행하는 기능
foods.foreach(lambda x : print(x))

파스타
피자
샐러드
서브웨이
닭갈비


# Narrow Operation 1:1 연산 

filter,map,flatMap,sample,union

In [18]:
sample_rdd = spark.parallelize([1,2,3,4,5])

In [21]:
sample_rdd2 = sample_rdd.map(lambda x : x+2)

In [22]:
sample_rdd2.collect()

[3, 4, 5, 6, 7]

In [24]:
movies = [
    "그린 북",
    "매트릭스",
    "토이 스토리",
    "캐스트 어웨이",
    "포드 V 페라리",
    "보헤미안 랩소디",
    "빽 투 더 퓨처",
    "반지의 제왕",
    "죽은 시인의 사회"
]

In [25]:
moviesRDD = spark.parallelize(movies)

In [28]:
mapmovies = moviesRDD.map(lambda x : x.split(" "))
mapmovies

PythonRDD[23] at RDD at PythonRDD.scala:53

In [29]:
mapmovies.collect()

[['그린', '북'],
 ['매트릭스'],
 ['토이', '스토리'],
 ['캐스트', '어웨이'],
 ['포드', 'V', '페라리'],
 ['보헤미안', '랩소디'],
 ['빽', '투', '더', '퓨처'],
 ['반지의', '제왕'],
 ['죽은', '시인의', '사회']]

In [37]:
flatMapMovies = moviesRDD.flatMap(lambda x:x.split(" "))
flatMapMovies.collect()

['그린',
 '북',
 '매트릭스',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

In [35]:
filteredMovies= flatMapMovies.filter(lambda x: x!="매트릭스")
filteredMovies.collect()

['그린',
 '북',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

In [38]:
#집합연산
num1 = spark.parallelize([1,2,3,4,5])
num2 = spark.parallelize([4,5,6,7,8,9,10])

In [40]:
#intersection 교집합만 추출
num1.intersection(num2).collect()

[4, 5]

In [42]:
#union 중복 포함 합
num1.union(num2).collect()

[1, 2, 3, 4, 5, 4, 5, 6, 7, 8, 9, 10]

In [43]:
#subtract
num1.subtract(num2).collect()

[2, 1, 3]

# sample(withReplacement,fraction,seed)
일부를 샘플링해서 추출
withReplacement : 비복원,복원 추출
fraction : 기댓값
seed : 난수추출을 위한 시드값 - 재현 가능

In [52]:
numlist = num1.union(num2)
numlist.collect()

[1, 2, 3, 4, 5, 4, 5, 6, 7, 8, 9, 10]

In [53]:
numlist.sample(True, 0.3).collect()

[4, 8, 9]

In [56]:
numlist.sample(False, 0.7).collect() # 재현 안됨

[1, 4, 5, 6, 7, 8, 9]

In [57]:
numlist.sample(True, 0.5, seed=42).collect() #재현 가능 시드O 

[5, 6, 6]

wide transformation

groupby reduce

In [59]:
foods.collect()

['파스타', '피자', '샐러드', '서브웨이', '닭갈비']

In [63]:
foodsGroup = foods.groupBy(lambda x:x[0])
res = foodsGroup.collect()

In [64]:
for (k,v) in res:
    print(k, list(v))

파 ['파스타']
피 ['피자']
샐 ['샐러드']
서 ['서브웨이']
닭 ['닭갈비']


In [65]:
spark.stop()